# Exploratory Data Analysis (EDA) on Large-Scale Operational Data

This notebook contains an exploratory data analysis (EDA) of a dataset related to operational processes, focusing on data cleaning, anomaly detection, and statistical insights.

## Data Cleaning and Preprocessing

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import json
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

#-----------READ DATA-------------
parquet_folder = "/Users/Documents/data"
parquet_file = ["DataTEC-11-01-to-11-30", "DataTEC-01-01-to-01-31"]
file_path = f"{parquet_folder}/{parquet_file[0]}"  # Nov
file_path2 = f"{parquet_folder}/{parquet_file[1]}"  # Jan

df = pq.read_table(file_path).to_pandas()
df2 = pq.read_table(file_path2).to_pandas()

print("The dataframe has: ", df.shape[0], "observations")
print("The dataframe has: ", df2.shape[0], "observations")

df_total = pd.concat([df, df2], ignore_index=True)

df_total.head(5)  
df_total.tail(5)  

print("The dataframe has: ", df_total.shape[0], "observations")

# Filtered variables list
variables_filtradas = [
    # Energy
    "CONTIFORM_MMA.CONTIFORM_MMA1.EnergyState.0",
    "CONTIFORM_MMA.CONTIFORM_MMA1.EnergyMeasurement_kWh_ElectPower_MainMachine.0",
    "CONTIFORM_MMA.CONTIFORM_MMA1.ElectricalData_UPSPower.0",
    "CONTIFORM_MMA.CONTIFORM_MMA1.ElectricalData_MainMachine.0",

    # Preform temperature in different layers
    "CONTIFORM_MMA.CONTIFORM_MMA1.PreformTemperatureLayer.1",
    "CONTIFORM_MMA.CONTIFORM_MMA1.PreformTemperatureLayer.3",
    "CONTIFORM_MMA.CONTIFORM_MMA1.PreformTemperatureLayer.5",
    "CONTIFORM_MMA.CONTIFORM_MMA1.PreformTemperatureLayer.7",
    "CONTIFORM_MMA.CONTIFORM_MMA1.PreformTemperatureLayer.9",

    # Other relevant temperatures
    "CONTIFORM_MMA.CONTIFORM_MMA1.CurrentTemperatureRotaryJoint.0",
    "CONTIFORM_MMA.CONTIFORM_MMA1.CurrentPreformNeckFinishTemperature.0",
    "CONTIFORM_MMA.CONTIFORM_MMA1.CurrentTemperatureBrake.1",
    "CONTIFORM_MMA.CONTIFORM_MMA1.CurrentTemperatureBrake.2",
    "CONTIFORM_MMA.CONTIFORM_MMA1.CurrentPreformTemperatureOvenInfeed.0",
    "CONTIFORM_MMA.CONTIFORM_MMA1.ActualTemperatureCoolingCircuit2.0",
    "CONTIFORM_MMA.CONTIFORM_MMA1.CurrentTemperaturePressureDewPoint.0",

    # Cooling and air control
    "CONTIFORM_MMA.CONTIFORM_MMA1.CoolingAirTemperatureActualValue.0",
    "CONTIFORM_MMA.CONTIFORM_MMA1.ContollerFactorCoolingCircuit1.0",
    "CONTIFORM_MMA.CONTIFORM_MMA1.AirWizardBasicController.0",
    "CONTIFORM_MMA.CONTIFORM_MMA1.AirWizardPlusController.0",

    # Pressure
    "CONTIFORM_MMA.CONTIFORM_MMA1.PressureCompensationChamberPressureActualValue.0",
    "CONTIFORM_MMA.CONTIFORM_MMA1.FinalBlowingPressureActualValue.0",

    # Speed
    "CONTIFORM_MMA.CONTIFORM_MMA1.WS_Cur_Mach_Spd.0",
    "CONTIFORM_MMA.CONTIFORM_MMA1.BeltDriveSpeedSetPoint.0",

    # Production and rejections
    "CONTIFORM_MMA.CONTIFORM_MMA1.WS_Tot_Rej.0",
    "CONTIFORM_MMA.CONTIFORM_MMA1.WS_Tot_Bottles.0",

    # Other operational parameters
    "CONTIFORM_MMA.CONTIFORM_MMA1.ActualHeightBaseCooling.0",
    "CONTIFORM_MMA.CONTIFORM_MMA1.CurrentProcessType_ConfigValue.0"
]

df_vars = df_total[df_total["variable"].isin(variables_filtradas)].copy()

print("The dataframe has: ", df_vars.shape[0], "observations")

# Percentage of the total observations represented by filtered data
df_vars.shape[0] / df_total.shape[0]

print("is a sample of ", round((df_vars.shape[0] / df_total.shape[0]) * 100), "% of the observations ")

#--------------------subtracting hours-------------------
# Convert 'user_ts' to datetime 
df_vars["user_ts"] = pd.to_datetime(df_vars["user_ts"])
# Subtract 6 hours
df_vars["user_ts"] = df_vars["user_ts"] - pd.Timedelta(hours=6)
df_vars.head(20)
df_vars.tail(20)

#----------------------MAKE THE DFs--------------------------
# Create a dictionary of dataframes for each variable
filtered_dfs = {var.replace(".", "_"): df_vars[df_vars["variable"] == var].copy() for var in variables_filtradas}
# Save the dataframes in individual variables
df_names = {}
for var, df_var in filtered_dfs.items():
    globals()[f"df_{var}"] = df_var
    df_names[f"df_{var}"] = df_var

print("DataFrames created:")
for name in df_names.keys():
    print(name)

# NUM OF OBSERVATIONS PER VARIABLE IN DF TOTAL 
df_observaciones = {name: df.shape[0] for name, df in filtered_dfs.items()}
df_observaciones_ordenado = dict(sorted(df_observaciones.items(), key=lambda item: item[1]))
df_observaciones_ordenado

# PERCENTAGE PER VARIABLE IN DF TOTAL 
proporcion_datos = {name: (df.shape[0] / df_total.shape[0]) * 100 for name, df in filtered_dfs.items()}
df_proporcion_ordenado = dict(sorted(proporcion_datos.items(), key=lambda item: item[1]))
df_proporcion_ordenado

# Variables to be discarded
variables_desechables = [ 
    'CONTIFORM_MMA_CONTIFORM_MMA1_CurrentProcessType_ConfigValue_0',
    'CONTIFORM_MMA_CONTIFORM_MMA1_ContollerFactorCoolingCircuit1_0',
    'CONTIFORM_MMA_CONTIFORM_MMA1_BeltDriveSpeedSetPoint_0',
    'CONTIFORM_MMA_CONTIFORM_MMA1_ActualHeightBaseCooling_0',
    'CONTIFORM_MMA_CONTIFORM_MMA1_WS_Cur_Mach_Spd_0',
    'CONTIFORM_MMA_CONTIFORM_MMA1_EnergyState_0',
    'CONTIFORM_MMA_CONTIFORM_MMA1_CoolingAirTemperatureActualValue_0',
    'CONTIFORM_MMA_CONTIFORM_MMA1_WS_Tot_Rej_0'
]

# ELIMINATE NON REPRESENTATIVE VARIABLES < 0.1%
for var in variables_desechables:
    filtered_dfs.pop(var, None)

print("Variables in filtered_dfs:")
for name in filtered_dfs.keys():
    print(name)

# ------------------- CREATE CHUNKS OF THE DFs ------------------
def chunk_dataframe(df, chunk_size=100000):
    """Divide DataFrame into smaller chunks."""
    num_chunks = len(df) // chunk_size + (1 if len(df) % chunk_size != 0 else 0)
    return [df.iloc[i * chunk_size:(i + 1) * chunk_size] for i in range(num_chunks)]

# Chunks for each df
chunked_dfs = {name: chunk_dataframe(df_var) for name, df_var in filtered_dfs.items()}

print("Chunks created per df:")
for name, chunks in chunked_dfs.items():
    print(f"{name}: {len(chunks)} chunks")

# ------------------- FUNCTIONS TO DETECT RELEVANCE AND RELATE ------------------
def calculate_variability(df, column):
    std_dev = df[column].std()
    iqr = df[column].quantile(0.75) - df[column].quantile(0.25)
    return {"Standard Deviation": std_dev, "IQR": iqr}

def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return len(outliers)

def calculate_data_proportion(df, column):
    return df[column].count() / df_total.shape[0]

def decompress_message_first_chunk(chunked_dfs):
    for name, chunks in chunked_dfs.items():
        if chunks and "message" in chunks[0].columns:  # Check if chunks exist and 'message' is in the first one
            chunk = chunks[0]

            # Filter non-null 'message' values
            if chunk["message"].notna().any():
                chunk = chunk.copy()  

                # Convert to JSON if it's a string
                chunk["message"] = chunk["message"].apply(lambda x: json.loads(x) if isinstance(x, str) else {})

                # Extract data
                extracted_columns = set()
                for message in chunk["message"]:
                    if isinstance(message, dict):  
                        extracted_columns.update(message.keys())

                # Create new columns and populate them with JSON values
                for col in extracted_columns:
                    chunk[col] = chunk["message"].apply(lambda x: x.get(col, None) if isinstance(x, dict) else None)

                # Drop the 'message' column after extracting information
                chunk.drop(columns=["message"], inplace=True)

                # Save the updated chunk in the respective dictionary
                chunked_dfs[name] = chunk

    return chunked_dfs

chunked_dfs = decompress_message_first_chunk(chunked_dfs)

print("Decompressed message data for relevant chunks.")

# Check the first 5 records of each first chunk in chunked_dfs
for name, chunks in chunked_dfs.items():
    if chunks:  # Check if there is at least one chunk available
        print(f"First chunk of {name}:")
        display(chunks[0].head(10))  # Show the first rows of the first chunk
        print("-" * 50)

#---------------APPLY FUNCTIONS TO THE CHUNK-----------------
# Function to apply to all chunks
def evaluate_all_chunks(chunked_dfs):
    """Applies relevant functions to all columns of the first chunks of each variable."""
    results = {}
    for name, chunks in chunked_dfs.items():
        if chunks:  # Check if there is at least one chunk available
            chunk = chunks[0]  # Take the first chunk
            results[name] = {}
            for column in chunk.columns:
                if chunk[column].dtype in [np.float64, np.int64]:  # Only analyze numeric variables
                    var_result = calculate_variability(chunk, column)
                    num_outliers = detect_outliers(chunk, column)
                    data_proportion = calculate_data_proportion(chunk, column)
                    results[name][column] = {
                        "Standard Deviation": var_result["Standard Deviation"],
                        "IQR": var_result["IQR"],
                        "Number of Outliers": num_outliers,
                        "Data Proportion": data_proportion
                    }
    return results

relevance_results = evaluate_all_chunks(chunked_dfs)
import pprint
pprint.pprint(relevance_results)

#--------------AFTER ANALYZING METRICS---------------------

# LOW VARIABILITY
low_variability_variables = [
    "CONTIFORM_MMA_CONTIFORM_MMA1_AirWizardBasicController_0", 
    "CONTIFORM_MMA_CONTIFORM_MMA1_AirWizardPlusController_0",
    "CONTIFORM_MMA_CONTIFORM_MMA1_ElectricalData_MainMachine_0.voltageMaxL1",
    "CONTIFORM_MMA_CONTIFORM_MMA1_ElectricalData_UPSPower_0"
]

# PLOT INDIVIDUAL GRAPHS ONE BY ONE
for var in low_variability_variables:
    for name, chunks in chunked_dfs.items():
        if name == var and chunks:
            chunk = chunks[0]  # First chunk
            for column in chunk.columns:
                if column not in ["user_ts", "variable"]: 
                    plt.figure(figsize=(10, 5))
                    if "user_ts" in chunk.columns:
                        plt.plot(chunk["user_ts"], chunk[column], marker='o', linestyle='-', alpha=0.5)
                        plt.xlabel("Time (user_ts)")
                    else:
                        plt.plot(chunk.index, chunk[column], marker='o', linestyle='-', alpha=0.5)
                        plt.xlabel("Index")
                    plt.title(f"Evolution of {column} in {var} (first chunk)")
                    plt.ylabel(column)
                    plt.xticks(rotation=45)
                    plt.grid(True)
                    plt.show()

# GROUPED PLOTS 
for var in low_variability_variables:
    for name, chunks in chunked_dfs.items():
        if name == var and chunks:
            chunk = chunks[0]  # First chunk
            plt.figure(figsize=(12, 6))
            for column in chunk.columns:
                if column not in ["user_ts", "variable"]: 
                    if "user_ts" in chunk.columns:
                        plt.plot(chunk["user_ts"], chunk[column], marker='o', linestyle='-', alpha=0.5, label=column)
                    else:
                        plt.plot(chunk.index, chunk[column], marker='o', linestyle='-', alpha=0.5, label=column)
            plt.xlabel("Time (user_ts)" if "user_ts" in chunk.columns else "Index")
            plt.title(f"Evolution of all columns in {var} (first chunk)")
            plt.ylabel("Value")
            plt.xticks(rotation=45)
            plt.grid(True)
            plt.legend()
            plt.show()


#--------------------DF TO CSV OF FINAL VARIABLES---------------------
'''
variables_to_export = [
    "CONTIFORM_MMA.CONTIFORM_MMA1.PreformTemperatureLayer.1",
    "CONTIFORM_MMA.CONTIFORM_MMA1.PreformTemperatureLayer.3",
    "CONTIFORM_MMA.CONTIFORM_MMA1.PreformTemperatureLayer.5",
    "CONTIFORM_MMA.CONTIFORM_MMA1.PreformTemperatureLayer.7",
    "CONTIFORM_MMA.CONTIFORM_MMA1.PreformTemperatureLayer.9",
    "CONTIFORM_MMA.CONTIFORM_MMA1.FinalBlowingPressureActualValue.0",
    "CONTIFORM_MMA.CONTIFORM_MMA1.PressureCompensationChamberPressureActualValue.0"
]

observations_per_df = {var: filtered_dfs[var.replace(".", "_")].shape[0] for var in variables_to_export if var.replace(".", "_") in filtered_dfs}

# Display the result
for var, num_obs in observations_per_df.items():
    print(f"{var}: {num_obs} observations")

for var in variables_to_export:
    formatted_var = var.replace(".", "_")  # Replace dots with underscores
    if formatted_var in filtered_dfs:  # Check if it exists in the DataFrames
        file_name = f"{formatted_var}.csv"
        filtered_dfs[formatted_var].to_csv(file_name, index=False)
        print(f"Saved: {file_name}")
'''

## Variable Exploration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

# Style
sns.set_style("whitegrid")

data_pressure_soplado = df_CONTIFORM_MMA_CONTIFORM_MMA1_FinalBlowingPressureActualValue_0.copy()
data_pressure_compensado = df_CONTIFORM_MMA_CONTIFORM_MMA1_PressureCompensationChamberPressureActualValue_0.copy()

#-------------------------------PRESSURE SOPLADO -------------------------
# ---- INITIAL DESCRIPTIVE ANALYSIS ----
print("Dataset information:")
print(data_pressure_soplado.info())
print("\nNull values:")
print(data_pressure_soplado.isnull().sum())
print("\nDuplicate values:")
print(data_pressure_soplado.duplicated().sum())
# data_pressure_soplado.drop(columns='message', inplace=True)
# STADISTIC SUMMARY
print("\nStatistical summary:")
print(data_pressure_soplado.describe(percentiles=[0.25, 0.5, 0.75]))
data_pressure_soplado.shape[0]
data_pressure_soplado.head(20)
data_pressure_compensado.shape[0]

# ---- DISTRIBUTIONS AND OUTLIERS DETECTION ----

col = 'value'
plt.figure(figsize=(5, 5))
plt.figure()
sns.histplot(data_pressure_soplado[col], kde=True, bins=20)
plt.title(f"Final Blowing Pressure Distribution")
plt.show()

plt.figure()
sns.boxplot(y=data_pressure_soplado[col])
plt.title(f"Final Blowing Pressure Boxplot")
plt.show()
print(col)

# IQR FOR OUTLIERS

Q1 = data_pressure_soplado[col].quantile(0.25)
Q3 = data_pressure_soplado[col].quantile(0.75)
IQR = Q3 - Q1
outliers = data_pressure_soplado[(data_pressure_soplado[col] < (Q1 - 1.5 * IQR)) | (data_pressure_soplado[col] > (Q3 + 1.5 * IQR))]
print(f"Outliers detected in {col} by IQR:")
print(outliers[[col]])
print(col)

# ---- TEMPORAL ANALYSIS AND TENDENCIES ----
data_pressure_soplado["user_ts"] = pd.to_datetime(data_pressure_soplado["user_ts"])
data_pressure_soplado = data_pressure_soplado.sort_values(by="user_ts")
plt.figure(figsize=(10, 4))
plt.plot(data_pressure_soplado["user_ts"], data_pressure_soplado[col], label=col)
plt.title(f"Time Series of {col}")
plt.legend()
plt.show()

plt.figure(figsize=(10, 4))
plt.plot(data_pressure_soplado["user_ts"], data_pressure_soplado[col].rolling(window=30).mean(), label="Rolling Mean")
plt.plot(data_pressure_soplado["user_ts"], data_pressure_soplado[col].rolling(window=30).std(), label="Rolling Std")
plt.title(f"Rolling Mean & Std of {col} FinalBlowingPressureActualValue")
plt.legend()
plt.show()

#-------------------------PRESSURE COMPENSADO--------------------
# --- INITIAL DESCRIPTIVE ANALYSIS ----
print("Dataset information:")
print(data_pressure_compensado.info())
print("\nNull values:")
print(data_pressure_compensado.isnull().sum())
print("\nDuplicate values:")
print(data_pressure_compensado.duplicated().sum())
# data_pressure_soplado.drop(columns='message', inplace=True)
# STATISTIC SUMMARY
print("\nStatistical summary:")
print(data_pressure_compensado.describe(percentiles=[0.25, 0.5, 0.75]))

# ----- DISTRIBUTIONS AND OUTLIERS DETECTION ----

col = 'value'
plt.figure(figsize=(5, 5))
plt.figure()
sns.histplot(data_pressure_compensado[col], kde=True, bins=10)
plt.title(f"Pressure Chamber Compensation Distribution:")
plt.show()

plt.figure()
sns.boxplot(y=data_pressure_compensado[col])
plt.title(f"Pressure Chamber Compensation Boxplot:")
plt.show()
print(col)

# IQR FOR OUTLIERS
Q1 = data_pressure_compensado[col].quantile(0.25)
Q3 = data_pressure_compensado[col].quantile(0.75)
IQR = Q3 - Q1
outliers = data_pressure_compensado[(data_pressure_compensado[col] < (Q1 - 1.5 * IQR)) | (data_pressure_soplado[col] > (Q3 + 1.5 * IQR))]
print(f"Outliers detected in {col} by IQR:")
print(outliers[[col]])
print(col)

# ---- TEMPORAL ANALYSIS AND TENDENCIES ----
data_pressure_compensado["user_ts"] = pd.to_datetime(data_pressure_compensado["user_ts"])
data_pressure_compensado = data_pressure_compensado.sort_values(by="user_ts")
plt.figure(figsize=(10, 4))
plt.plot(data_pressure_compensado["user_ts"], data_pressure_compensado[col], label=col)
plt.title(f"Time Series of {col}")
plt.legend()
plt.show()

plt.figure(figsize=(10, 4))
plt.plot(data_pressure_compensado["user_ts"], data_pressure_compensado[col].rolling(window=30).mean(), label="Rolling Mean")
plt.plot(data_pressure_compensado["user_ts"], data_pressure_compensado[col].rolling(window=30).std(), label="Rolling Std")
plt.title(f"Rolling Mean & Std of {col} FinalBlowingPressureActualValue")
plt.legend()
plt.show()

# TIME SERIES FILTERED
# PRESSURE CHAMBER

data_pressure_compensado["user_ts"] = pd.to_datetime(data_pressure_compensado["user_ts"])
# NOV 1- JAN 2
start_date = "2024-11-15"
end_date = "2025-12-01"
filtered_data = data_pressure_compensado[(data_pressure_compensado["user_ts"] >= start_date) & 
                                         (data_pressure_compensado["user_ts"] <= end_date)]

filtered_data = filtered_data.sort_values(by="user_ts")

plt.figure(figsize=(10, 4))
plt.plot(filtered_data["user_ts"], filtered_data[col], label=col)
plt.xlabel("Date")
plt.ylabel("Pressure Value")
plt.title(f"Time Series of Pressure in Compensation Chamber (Nov 15 - Dec 2)")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()

# PRESSURE SOPLADO
data_pressure_soplado["user_ts"] = pd.to_datetime(data_pressure_soplado["user_ts"])
start_date = "2024-11-15"
end_date = "2025-12-01"
filtered_data = data_pressure_soplado[(data_pressure_soplado["user_ts"] >= start_date) & 
                                         (data_pressure_soplado["user_ts"] <= end_date)]

filtered_data = filtered_data.sort_values(by="user_ts")

plt.figure(figsize=(10, 4))
plt.plot(filtered_data["user_ts"], filtered_data[col], label=col)
plt.xlabel("Date")
plt.ylabel("Pressure Value")
plt.title(f"Time Series of Blowing Pressure (Nov 11 - Dec 2)")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
#-------------CONTIFORM_MMA_CONTIFORM_MMA1_WS_Tot_Rej_0.csv------------------

# ---- 1.1. Initial Descriptive Analysis ----
print("Dataset information:")
print(data.info())
print("\nNull values:")
print(data.isnull().sum())
print("\nDuplicate values:")
print(data.duplicated().sum())

# Statistical summary
print("\nStatistical summary:")
print(data.describe(percentiles=[0.25, 0.5, 0.75]))

# ---- 1.2. Distribution Analysis and Outlier Detection ----
plt.figure(figsize=(10, 5))

for col in data.select_dtypes(include=np.number).columns:
    col_data = data[col].dropna()  # Remove NaN values
    
    if col_data.nunique() > 1:  # Ensure there is more than one unique value
        # Histogram and KDE
        plt.figure()
        sns.histplot(col_data, kde=True, bins=30)
        plt.title(f"Distribution of {col}")
        plt.show()

        # Boxplot
        plt.figure()
        sns.boxplot(y=col_data)
        plt.title(f"Boxplot of {col}")
        plt.show()
    else:
        print(f"The column '{col}' has insufficient values for plotting.")

    # Z-score to detect outliers
    z_scores = np.abs(zscore(col_data))
    outliers_z = col_data[z_scores > 3]
    
    print(f"\nOutliers detected in '{col}' (Z-score > 3):")
    if not outliers_z.empty:
        print(outliers_z)
    else:
        print("No outliers found with Z-score.")

# IQR (Interquartile Range method) to detect outliers
for col in data.select_dtypes(include=np.number).columns:
    col_data = data[col].dropna()  # Remove NaN values

    Q1 = col_data.quantile(0.25)
    Q3 = col_data.quantile(0.75)
    IQR = Q3 - Q1

    outliers_iqr = col_data[(col_data < (Q1 - 1.5 * IQR)) | (col_data > (Q3 + 1.5 * IQR))]

    print(f"\nOutliers detected in '{col}' using IQR:")
    if not outliers_iqr.empty:
        print(outliers_iqr)
    else:
        print("No outliers found with IQR.")


# ---- 1.3. Temporal Analysis and Trends ----
if "user_ts" in data.columns:
    print("\nThe 'user_ts' column exists. Converting to datetime format...\n")
    
    data["user_ts"] = pd.to_datetime(data["user_ts"], errors="coerce")  # Error handling
    data = data.dropna(subset=["user_ts"])  # Remove NaN values in dates
    data = data.sort_values(by="user_ts")

    print(data.info())  # Debugging: check if conversion was successful
    
    # Plotting time series
    for col in data.select_dtypes(include=np.number).columns:
        if data[col].dropna().empty:  # Avoid plotting columns with no valid data
            print(f"The column '{col}' has no valid numerical data.")
            continue
        
        plt.figure(figsize=(10, 4))
        plt.plot(data["user_ts"], data[col], label=col)
        plt.title(f"Time series of {col}")
        plt.xlabel("Date")
        plt.ylabel(col)
        plt.legend()
        plt.grid()
        plt.show(block=True)  # Ensure the plot is shown

    # Rolling Mean & Std
    for col in data.select_dtypes(include=np.number).columns:
        if len(data[col].dropna()) < 30:  # Adjust window size if there is little data
            window_size = max(1, len(data[col].dropna()) // 2)  # Adjust to half if there is little data
            print(f"Adjusting rolling window to {window_size} for '{col}' (insufficient data).")
        else:
            window_size = 30

        plt.figure(figsize=(10, 4))
        plt.plot(data["user_ts"], data[col].rolling(window=window_size).mean(), label="Rolling Mean", color="blue")
        plt.plot(data["user_ts"], data[col].rolling(window=window_size).std(), label="Rolling Std", color="red")
        plt.title(f"Rolling Mean & Std of {col}")
        plt.xlabel("Date")
        plt.legend()
        plt.grid()
        plt.show(block=True)
else:
    print("\nThe 'user_ts' column does not exist in the DataFrame.\n")
# Calculate Q1, Q3, and IQR
Q1 = data['value'].quantile(0.25)
Q3 = data['value'].quantile(0.75)
IQR = Q3 - Q1

# Define the limits
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Check the limits
print(f"Lower bound: {lower_bound}")
print(f"Upper bound: {upper_bound}")

# Count values near the limits
print(f"Values less than {lower_bound + 0.1 * IQR}: {sum(data['value'] < (lower_bound + 0.1 * IQR))}")
print(f"Values greater than {upper_bound - 0.1 * IQR}: {sum(data['value'] > (upper_bound - 0.1 * IQR))}")

# Adjust the criteria if necessary
lower_bound_adj = Q1 - 2.0 * IQR
upper_bound_adj = Q3 + 2.0 * IQR

anomalies = data[(data['value'] < lower_bound_adj) | (data['value'] > upper_bound_adj)]

print("Detected anomalies (adjusted criteria):")
print(anomalies)
plt.figure(figsize=(12, 5))
plt.plot(data["user_ts"], data["value"], marker="o", linestyle="-")
plt.title("Evolution of 'value' over time")
plt.xticks(rotation=45)
plt.show()

# Specify the exact date and time to analyze
start_datetime = "2024-11-15 12:20:00"
end_datetime = "2024-12-01 12:45:00"

# Filter data within that interval
filtered_df = data[(data["user_ts"] >= start_datetime) & (data["user_ts"] <= end_datetime)]

# Verify filtered data
print("Filtered data in the selected interval:")
print(filtered_df.head())

# Analyze basic statistics of behavior during that time
print("\nStatistical summary:")
print(filtered_df.describe())

# Plot evolution of variables over time within that interval
plt.figure(figsize=(10, 4))
for col in filtered_df.select_dtypes(include="number").columns:
    plt.plot(filtered_df["user_ts"], filtered_df[col], label=col)

plt.title(f"Behavior in the interval {start_datetime} - {end_datetime}")
plt.legend()
plt.xticks(rotation=45)
plt.show()

#-------------CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_7----------------

# Load the CSV file
file = "/Users/Documents/CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_7.csv"  
df = pd.read_csv(file)

# Convert the time column to datetime format
df["user_ts"] = pd.to_datetime(df["user_ts"], format="ISO8601", errors="coerce")

# Specify the exact date and time to analyze
start_datetime = "2024-11-01 12:20:00"
end_datetime = "2025-02-01 12:45:00"

# Filter data within that interval
filtered_df = df[(df["user_ts"] >= start_datetime) & (df["user_ts"] <= end_datetime)]

# Verify filtered data
print("Filtered data in the selected interval:")
print(filtered_df.head())

# Analyze basic statistics of behavior during that time
print("\nStatistical summary:")
print(filtered_df.describe())

# Plot evolution of variables over time within that interval
plt.figure(figsize=(10, 4))
for col in filtered_df.select_dtypes(include="number").columns:
    plt.plot(filtered_df["user_ts"], filtered_df[col], label=col)

plt.title(f"Behavior in the interval {start_datetime} - {end_datetime}")
plt.legend()
plt.xticks(rotation=45)
plt.show()

numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns

# Calculate statistics for each numeric column
statistics = df[numeric_columns].describe(percentiles=[0.25, 0.5, 0.75]).T
statistics["std"] = df[numeric_columns].std()  # Add standard deviation

# Show results
print("Statistical summary of numeric variables:")
print(statistics)

# Total number of records
total_records = df.shape[0]

# Null values per column
null_values = df.isnull().sum()

# Duplicated values
duplicated_values = df.duplicated().sum()

# Show results
print(f"Total records: {total_records}")
print(f"Null values per column:\n{null_values}")
print(f"Total duplicated values: {duplicated_values}")

# ---- 1.2. Distribution Analysis and Outlier Detection ----
plt.figure(figsize=(10, 5))

for col in df.select_dtypes(include=np.number).columns:
    col_df = df[col].dropna()  # Remove NaN values
    
    if col_df.nunique() > 1:  # Ensure there is more than one unique value
        # Histogram and KDE
        plt.figure()
        sns.histplot(col_df, kde=True, bins=30)
        plt.title(f"Distribution of {col}")
        plt.show()

        # Boxplot
        plt.figure()
        sns.boxplot(y=col_df)
        plt.title(f"Boxplot of {col}")
        plt.show()
    else:
        print(f"The column '{col}' has insufficient values for plotting.")

Q1 = filtered_df['value'].quantile(0.25)
Q3 = filtered_df['value'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier limits
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
anomalies = filtered_df[(filtered_df['value'] < lower_bound) | (filtered_df['value] > upper_bound)]

print("Anomalies detected:")
print(anomalies)

# IQR (Interquartile Range method) to detect outliers
for col in df.select_dtypes(include=np.number).columns:
    col_df = df[col].dropna()  # Remove NaN values

    Q1 = col_df.quantile(0.25)
    Q3 = col_df.quantile(0.75)
    IQR = Q3 - Q1

    outliers_iqr = col_df[(col_df < (Q1 - 1.5 * IQR)) | (col_df > (Q3 + 1.5 * IQR))]

    print(f"\nOutliers detected in '{col}' using IQR:")
    if not outliers_iqr.empty:
        print(outliers_iqr)
    else:
        print("No outliers found using IQR.")  # Specify the exact date and time for analysis

start_datetime = "2024-11-15 12:20:00"
end_datetime = "2025-12-01 12:45:00"

# Filter data within that interval
filtered_df = df[(df["user_ts"] >= fecha_hora_inicio) & (df["user_ts"] <= fecha_hora_fin)]

# Verify filtered data
print("Filtered data in the selected interval:")
print(filtered_df.head())

# Analyze basic statistics of the behavior within that time
print("\nStatistical Summary:")
print(filtered_dfo.describe())

# Plot evolution of variables over time within that interval
plt.figure(figsize=(10, 4))
for col in filtered_df.select_dtypes(include="number").columns:
    plt.plot(filtered_df["user_ts"], filtered_df[col], label=col)

plt.title(f"Behavior in the interval {start_datetime} - {end_datetime}")
plt.legend()
plt.xticks(rotation=45)
plt.show()

# Rolling Mean & Std
for col in df.select_dtypes(include=np.number).columns:
        if len(df[col].dropna()) < 30:  # Adjust the window size if there are few data points
            window_size = max(1, len(df[col].dropna()) // 2)  # Adjust to half if there are few data points
            print(f"Warning: Adjusting rolling window to {window_size} for '{col}' (few data).")
        else:
            window_size = 30

        plt.figure(figsize=(10, 4))
        plt.plot(df["user_ts"], df[col].rolling(window=window_size).mean(), label="Rolling Mean", color="blue")
        plt.plot(df["user_ts"], df[col].rolling(window=window_size).std(), label="Rolling Std", color="red")
        plt.title(f"Rolling Mean & Std of {col}")
        plt.xlabel("Date")
        plt.legend()
        plt.grid()
        plt.show(block=True)

In [ ]:

# ----------------------------------------------------------------------Layer9.ipynb---------------------------------------------------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LinearRegression
from sklearn.cluster import DBSCAN
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import zscore

# Style configuration
sns.set_style("whitegrid")

# CSV file path
csv_file = "CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_9.csv"  # MODIFY THIS

# Load the CSV
data = pd.read_csv(csv_file)
print(data)

# ---- 1.1. Initial Descriptive Analysis ----
print("Dataset Information:")
print(data.info())
print("\nNull Values:")
print(data.isnull().sum())
print("\nDuplicate Values:")
print(data.duplicated().sum())

data.dropna(subset=['value'], inplace=True)
data.drop(columns=['message'], inplace=True)

data

# Statistical summary
print("\nStatistical Summary:")
print(data.describe(percentiles=[0.25, 0.5, 0.75]))

# ---- 1.2. Distribution Analysis and Outlier Detection ----
plt.figure(figsize=(10, 5))
for col in data.select_dtypes(include=np.number).columns:
    plt.figure()
    sns.histplot(data[col], kde=True, bins=30)
    plt.title(f"Distribution of {col}")
    plt.show()

    plt.figure()
    sns.boxplot(y=data[col])
    plt.title(f"Boxplot of {col}")
    plt.show()

# Z-score for outlier detection
    data[f"{col}_zscore"] = np.abs(zscore(data[col]))
    print(f"Outliers detected in {col} (Z-score > 3):")
    print(data[data[f"{col}_zscore"] > 3][[col]])

# IQR (Interquartile Range Method) for outlier detection
for col in data.select_dtypes(include=np.number).columns:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = data[(data[col] < (Q1 - 1.5 * IQR)) | (data[col] > (Q3 + 1.5 * IQR))]
    print(f"Outliers detected in {col} by IQR:")
    print(outliers[[col]])

if "user_ts" in data.columns:
    # The format string now matches the timestamp format in your data
    data["user_ts"] = pd.to_datetime(data["user_ts"], format='%Y-%m-%d %H:%M:%S%z', errors='coerce')
    data = data.dropna(subset=['user_ts'])
    data = data.sort_values(by="user_ts")
    for col in data.select_dtypes(include=np.number).columns:
        plt.figure(figsize=(10, 4))
        plt.plot(data["user_ts"], data[col], label=col)
        plt.title(f"Time Series of {col}")
        plt.legend()
        plt.show()

# Rolling Mean & Std
    for col in data.select_dtypes(include=np.number).columns:
        plt.figure(figsize=(10, 4))
        plt.plot(data["user_ts"], data[col].rolling(window=30).mean(), label="Rolling Mean")
        plt.plot(data["user_ts"], data[col].rolling(window=30).std(), label="Rolling Std")
        plt.title(f"Rolling Mean & Std of {col}")
        plt.legend()
        plt.show()

# ---- 1.4. Model-Based Anomaly Detection ----
X = data.select_dtypes(include=np.number)  # Only numeric variables
if not X.empty:
    X = X.dropna()
    # Linear regression
    if X.shape[1] > 1:
        reg = LinearRegression()
        reg.fit(X.iloc[:, :-1], X.iloc[:, -1])
        print("Linear regression coefficients:")
        print(reg.coef_)

    # DBSCAN for anomaly detection
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    data["dbscan_anomaly"] = dbscan.fit_predict(X)

    # Isolation Forest
    iso_forest = IsolationForest(contamination=0.05, random_state=42)
    data["isolation_anomaly"] = iso_forest.fit_predict(X)

    # LOF (Local Outlier Factor)
    lof = LocalOutlierFactor(n_neighbors=20)
    data["lof_anomaly"] = lof.fit_predict(X)

    # ---- Visualize anomalies ----
    plt.figure(figsize=(8, 5))
    sns.scatterplot(x=X.iloc[:, 0], y=X.iloc[:, 1] if X.shape[1] > 1 else X.iloc[:, 0], hue=data["isolation_anomaly"], palette={1:"blue", -1:"red"})
    plt.title("Anomalies detected with Isolation Forest")
    plt.legend(title="Anomaly", labels=["Normal", "Anomalous"])
    plt.show()

    print("\nNumber of anomalies detected by each method:")
    print("Isolation Forest:")
    print(data["isolation_anomaly"].value_counts())
    print("DBSCAN:")
    print(data["dbscan_anomaly"].value_counts())
    print("Local Outlier Factor (LOF):")
    print(data["lof_anomaly"].value_counts())


# -------------------------------------------------------------------Layer1.ipynb-------------------------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LinearRegression
from sklearn.cluster import DBSCAN
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import zscore

# Style configuration
sns.set_style("whitegrid")

# CSV file path
csv_file = "CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_1.csv"  # MODIFY THIS

# Load the CSV
data = pd.read_csv(csv_file)
print(data)

# ---- 1.1. Initial Descriptive Analysis ----
print("Dataset Information:")
print(data.info())
print("\nNull Values:")
print(data.isnull().sum())
print("\nDuplicate Values:")
print(data.duplicated().sum())

"""Remove null values from 'value' and remove the 'message' column as it contains only null values."""

data.dropna(subset=['value'], inplace=True)
data.drop(columns=['message'], inplace=True)

data

# Statistical summary
print("\nStatistical Summary:")
print(data.describe(percentiles=[0.25, 0.5, 0.75]))

# ---- 1.2. Distribution Analysis and Outlier Detection ----
plt.figure(figsize=(10, 5))
for col in data.select_dtypes(include=np.number).columns:
    plt.figure()
    sns.histplot(data[col], kde=True, bins=30)
    plt.title(f"Distribution of {col}")
    plt.show()

    plt.figure()
    sns.boxplot(y=data[col])
    plt.title(f"Boxplot of {col}")
    plt.show()

# Z-score for outlier detection
    data[f"{col}_zscore"] = np.abs(zscore(data[col]))
    print(f"Outliers detected in {col} (Z-score > 3):")
    print(data[data[f"{col}_zscore"] > 3][[col]])

# IQR (Interquartile Range Method) for outlier detection
for col in data.select_dtypes(include=np.number).columns:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = data[(data[col] < (Q1 - 1.5 * IQR)) | (data[col] > (Q3 + 1.5 * IQR))]
    print(f"Outliers detected in {col} by IQR:")
    print(outliers[[col]])

if "user_ts" in data.columns:
    # The format string now matches the timestamp format in your data
    data["user_ts"] = pd.to_datetime(data["user_ts"], format='%Y-%m-%d %H:%M:%S%z', errors='coerce')
    data = data.dropna(subset=['user_ts'])
    data = data.sort_values(by="user_ts")
    for col in data.select_dtypes(include=np.number).columns:
        plt.figure(figsize=(10, 4))
        plt.plot(data["user_ts"], data[col], label=col)
        plt.title(f"Time Series of {col}")
        plt.legend()
        plt.show()

# Rolling Mean & Std
    for col in data.select_dtypes(include=np.number).columns:
        plt.figure(figsize=(10, 4))
        plt.plot(data["user_ts"], data[col].rolling(window=30).mean(), label="Rolling Mean")
        plt.plot(data["user_ts"], data[col].rolling(window=30).std(), label="Rolling Std")
        plt.title(f"Rolling Mean & Std of {col}")
        plt.legend()
        plt.show()

# ---- 1.4. Model-Based Anomaly Detection ----
X = data.select_dtypes(include=np.number)  # Only numeric variables
if not X.empty:
    X = X.dropna()
    # Linear regression
    if X.shape[1] > 1:
        reg = LinearRegression()
        reg.fit(X.iloc[:, :-1], X.iloc[:, -1])
        print("Linear regression coefficients:")
        print(reg.coef_)

    # DBSCAN for anomaly detection
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    data["dbscan_anomaly"] = dbscan.fit_predict(X)

    # Isolation Forest
    iso_forest = IsolationForest(contamination=0.05, random_state=42)
    data["isolation_anomaly"] = iso_forest.fit_predict(X)

    # LOF (Local Outlier Factor)
    lof = LocalOutlierFactor(n_neighbors=20)
    data["lof_anomaly"] = lof.fit_predict(X)

    # ---- Visualize anomalies ----
    plt.figure(figsize=(8, 5))
    sns.scatterplot(x=X.iloc[:, 0], y=X.iloc[:, 1] if X.shape[1] > 1 else X.iloc[:, 0], hue=data["isolation_anomaly"], palette={1:"blue", -1:"red"})
    plt.title("Anomalies detected with Isolation Forest")
    plt.legend(title="Anomaly", labels=["Normal", "Anomalous"])
    plt.show()

    print("\nNumber of anomalies detected by each method:")
    print("Isolation Forest:")
    print(data["isolation_anomaly"].value_counts())
    print("DBSCAN:")
    print(data["dbscan_anomaly"].value_counts())
    print("Local Outlier Factor (LOF):")
    print(data["lof_anomaly"].value_counts())

In [ ]:
layer_5 = pd.read_csv("PreformTemperatureLayer_5.csv")
layer_3 = pd.read_csv("PreformTemperatureLayer_3.csv")

# ---- 1.1. Initial Descriptive Analysis ----
print("Dataset information for layer 3:")
print(layer_3.info())
print("Dataset information for layer 5:")
print(layer_5.info())

print("\nNull values in layer 3:")
print(layer_3.isnull().sum())
print("\nNull values in layer 5:")
print(layer_5.isnull().sum())

print("\nDuplicate values in layer 3:")
print(layer_3.duplicated().sum())
print("\nDuplicate values in layer 5:")
print(layer_5.duplicated().sum())

# Drop the 'message' column as it is completely null
layer_3.drop(columns=['message'], inplace=True)
layer_5.drop(columns=['message'], inplace=True)

# Handle null values in 'value'
if layer_3['value'].isnull().sum() > 0:
    layer_3['value'].fillna(layer_3['value'].median(), inplace=True)

if layer_5['value'].isnull().sum() > 0:
    layer_5['value'].fillna(layer_5['value'].median(), inplace=True)

print("\nStatistical summary of layer 3:")
print(layer_3.describe(percentiles=[0.25, 0.5, 0.75]))
print("\nStatistical summary of layer 5:")
print(layer_5.describe(percentiles=[0.25, 0.5, 0.75]))

# ---- 1.2. Distribution Analysis and Outlier Detection ----
plt.figure(figsize=(10, 5))
for col in layer_3.select_dtypes(include=np.number).columns:
    plt.figure()
    sns.histplot(layer_3[col], kde=True, bins=30)
    plt.title(f"Distribution of {col} in layer 3")
    plt.show()

    plt.figure()
    sns.boxplot(y=layer_3[col])
    plt.title(f"Boxplot of {col} in layer 3")
    plt.show()

    # Z-score for detecting outliers
    layer_3[f"{col}_zscore"] = np.abs(zscore(layer_3[col]))
    print(f"Outliers detected in {col} (Z-score > 3) in layer 3:")
    print(layer_3[layer_3[f"{col}_zscore"] > 3][[col]])

plt.figure(figsize=(10, 5))
for col in layer_5.select_dtypes(include=np.number).columns:
    plt.figure()
    sns.histplot(layer_5[col], kde=True, bins=30)
    plt.title(f"Distribution of {col} in layer 5")
    plt.show()

    plt.figure()
    sns.boxplot(y=layer_5[col])
    plt.title(f"Boxplot of {col} in layer 5")
    plt.show()

    # Z-score for detecting outliers
    layer_5[f"{col}_zscore"] = np.abs(zscore(layer_5[col]))
    print(f"Outliers detected in {col} (Z-score > 3) in layer 5:")
    print(layer_5[layer_5[f"{col}_zscore"] > 3][[col]])

# ---- 1.3. Temporal Analysis and Trends ----
if "user_ts" in layer_3.columns:
    layer_3["user_ts"] = pd.to_datetime(layer_3["user_ts"], format='%Y-%m-%d %H:%M:%S%z', errors='coerce')
    layer_3 = layer_3.dropna(subset=['user_ts'])
    layer_3 = layer_3.sort_values(by="user_ts")
    for col in layer_3.select_dtypes(include=np.number).columns:
        plt.figure(figsize=(10, 4))
        plt.plot(layer_3["user_ts"], layer_3[col], label=col)
        plt.title(f"Time series of {col} in layer 3")
        plt.legend()
        plt.show()

    # Rolling Mean & Std
    for col in layer_3.select_dtypes(include=np.number).columns:
        plt.figure(figsize=(10, 4))
        plt.plot(layer_3["user_ts"], layer_3[col].rolling(window=30).mean(), label="Rolling Mean")
        plt.plot(layer_3["user_ts"], layer_3[col].rolling(window=30).std(), label="Rolling Std")
        plt.title(f"Rolling Mean & Std of {col} in layer 3")
        plt.legend()
        plt.show()

if "user_ts" in layer_5.columns:
    layer_5["user_ts"] = pd.to_datetime(layer_5["user_ts"], format="mixed", errors="coerce")
    layer_5 = layer_5.dropna(subset=['user_ts'])
    layer_5 = layer_5.sort_values(by="user_ts")
    for col in layer_5.select_dtypes(include=np.number).columns:
        plt.figure(figsize=(10, 4))
        plt.plot(layer_5["user_ts"], layer_5[col], label=col)
        plt.title(f"Time series of {col} in layer 5")
        plt.legend()
        plt.show()

    # Rolling Mean & Std
    for col in layer_5.select_dtypes(include=np.number).columns:
        plt.figure(figsize=(10, 4))
        plt.plot(layer_5["user_ts"], layer_5[col].rolling(window=30).mean(), label="Rolling Mean")
        plt.plot(layer_5["user_ts"], layer_5[col].rolling(window=30).std(), label="Rolling Std")
        plt.title(f"Rolling Mean & Std of {col} in layer 5")
        plt.legend()
        plt.show()

# ---- 1.4. Model-Based Anomaly Detection ----
X = layer_3.select_dtypes(include=np.number)  # Only numeric variables
if not X.empty:

    # Linear Regression
    if X.shape[1] > 1:
        reg = LinearRegression()
        reg.fit(X.iloc[:, :-1], X.iloc[:, -1])
        print("Linear regression coefficients for layer 3:")
        print(reg.coef_)

    # DBSCAN for anomaly detection
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    layer_3["dbscan_anomaly"] = dbscan.fit_predict(X)

    # Isolation Forest
    iso_forest = IsolationForest(contamination=0.05, random_state=42)
    layer_3["isolation_anomaly"] = iso_forest.fit_predict(X)

    # LOF (Local Outlier Factor)
    lof = LocalOutlierFactor(n_neighbors=20)
    layer_3["lof_anomaly"] = lof.fit_predict(X)

    # ---- Visualize anomalies ----
    plt.figure(figsize=(8, 5))
    sns.scatterplot(x=X.iloc[:, 0], y=X.iloc[:, 1] if X.shape[1] > 1 else X.iloc[:, 0], hue=layer_3["isolation_anomaly"], palette={1:"blue", -1:"red"})
    plt.title("Anomalies detected with Isolation Forest in layer 3")
    plt.legend(title="Anomaly Layer 3", labels=["Normal", "Anomalous"])
    plt.show()

    print("\nNumber of anomalies detected by each method in layer 3:")
    print("Isolation Forest:")
    print(layer_3["isolation_anomaly"].value_counts())
    print("DBSCAN:")
    print(layer_3["dbscan_anomaly"].value_counts())
    print("Local Outlier Factor (LOF):")
    print(layer_3["lof_anomaly"].value_counts())


from sklearn.cluster import MiniBatchKMeans

# Select only numeric variables
X = layer_5.select_dtypes(include=np.number)  # Only numeric variables

# Reduce sample if dataset is too large
sample_size = min(50000, len(X))  # Take a maximum of 50,000 rows
X_sample = X.sample(n=sample_size, random_state=42)

if not X_sample.empty:
    # Linear Regression
    if X_sample.shape[1] > 1:
        reg = LinearRegression()
        reg.fit(X_sample.iloc[:, :-1], X_sample.iloc[:, -1])
        print("Linear regression coefficients for layer 5:")
        print(reg.coef_)

    # MiniBatchKMeans for clustering detection
    kmeans = MiniBatchKMeans(n_clusters=5, random_state=42, batch_size=10000)
    layer_5.loc[X_sample.index, "kmeans_cluster"] = kmeans.fit_predict(X_sample)

    # Isolation Forest
    iso_forest = IsolationForest(contamination=0.05, random_state=42)
    layer_5.loc[X_sample.index, "isolation_anomaly"] = iso_forest.fit_predict(X_sample)

    # LOF (Local Outlier Factor)
    lof = LocalOutlierFactor(n_neighbors=20)
    layer_5.loc[X_sample.index, "lof_anomaly"] = lof.fit_predict(X_sample)

    # ---- Visualize anomalies ----
    plt.figure(figsize=(8, 5))
    sns.scatterplot(x=X_sample.iloc[:, 0], y=X_sample.iloc[:, 1] if X_sample.shape[1] > 1 else X_sample.iloc[:, 0], hue=layer_5.loc[X_sample.index, "isolation_anomaly"], palette={1:"blue", -1:"red"})
    plt.title("Anomalies detected with Isolation Forest in layer 5 (Reduced Sample)")
    plt.legend(title="Anomaly Layer 5", labels=["Normal", "Anomalous"])
    plt.show()

    print("\nNumber of anomalies detected by each method in layer 5 (Reduced Sample):")
    print("Isolation Forest:")
    print(layer_5.loc[X_sample.index, "isolation_anomaly"].value_counts())
    print("MiniBatchKMeans Clusters:")
    print(layer_5.loc[X_sample.index, "kmeans_cluster"].value_counts())
    print("Local Outlier Factor (LOF):")
    print(layer_5.loc[X_sample.index, "lof_anomaly"].value_counts())

## Hypothesis Testing

In [ ]:

# -----------------------------------------------------------------Hypothesis1-----------------------------------------------------------------

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df_TEMP_LAYER_1 = pd.read_csv("CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_1.csv", index_col=0,parse_dates=True).drop(columns=["message"], errors="ignore").iloc[:57097]
df_TEMP_LAYER_3 = pd.read_csv("CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_3.csv", index_col=0,parse_dates=True).drop(columns=["message"], errors="ignore").iloc[:57097]
df_TEMP_LAYER_5 = pd.read_csv("CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_5.csv", index_col=0,parse_dates=True).drop(columns=["message"], errors="ignore").iloc[:57097]
df_TEMP_LAYER_7 = pd.read_csv("CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_7.csv", index_col=0,parse_dates=True).drop(columns=["message"], errors="ignore").iloc[:57097]
df_TEMP_LAYER_9 = pd.read_csv("CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_9.csv", index_col=0,parse_dates=True).drop(columns=["message"], errors="ignore").iloc[:57097]
df_REJECTS = pd.read_csv("CONTIFORM_MMA_CONTIFORM_MMA1_WS_Tot_Rej_0.csv", index_col=0,parse_dates=True).drop(columns=["message"], errors="ignore",inplace=True)

df_REJECTS = pd.read_csv("CONTIFORM_MMA_CONTIFORM_MMA1_WS_Tot_Rej_0.csv", index_col=0,parse_dates=True).drop(columns=["message"], errors="ignore")

# Assign the DataFrame with dropped columns to df_REJECTS.
df = df_REJECTS.copy()
df["PreformTemperatureLayer_1"] = df_TEMP_LAYER_1["value"] # Select the 'value' column
df["PreformTemperatureLayer_3"] = df_TEMP_LAYER_3["value"] # Select the 'value' column
df["PreformTemperatureLayer_5"] = df_TEMP_LAYER_5["value"] # Select the 'value' column
df["PreformTemperatureLayer_7"] = df_TEMP_LAYER_7["value"] # Select the 'value' column
df["PreformTemperatureLayer_9"] = df_TEMP_LAYER_9["value"] # Select the 'value' column

rejection_threshold = pd.to_numeric(df[df_REJECTS.columns[0]], errors='coerce').quantile(0.75)
df["Rejection Group"] = ["High" if x > rejection_threshold else "Low" for x in pd.to_numeric(df[df_REJECTS.columns[0]], errors='coerce')]

for var in ["PreformTemperatureLayer_1", "PreformTemperatureLayer_3", "PreformTemperatureLayer_5", "PreformTemperatureLayer_7", "PreformTemperatureLayer_9"]:
    plt.figure(figsize=(10, 5))
    sns.boxplot(x="Rejection Group", y=var, data=df)
    plt.title(f"Distribution of {var} in Lots with High and Low Rejection")
    plt.xlabel("Rejection Group")
    plt.ylabel("Preform Temperature")
    plt.show()
    
#-----------------------------------------------------Hypothesis2------------------------------------------------------------

'''
df_CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_1
df_CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_3
df_CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_5
df_CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_7
df_CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_9
df_CONTIFORM_MMA_CONTIFORM_MMA1_WS_Tot_Rej_0
'''


#-------------------Attempting to see if bottles per day increased------------------

dfs_a_graficar = {
    "Preform Temp Layer 1": df_CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_1,
    "Preform Temp Layer 3": df_CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_3,
    "Preform Temp Layer 5": df_CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_5,
    "Preform Temp Layer 7": df_CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_7,
    "Preform Temp Layer 9": df_CONTIFORM_MMA_CONTIFORM_MMA1_PreformTemperatureLayer_9
}



plt.figure(figsize=(12, 6))


fecha_inicio = pd.to_datetime("2024-11-15").tz_localize(None)
fecha_fin = pd.to_datetime("2025-12-01").tz_localize(None)

for nombre, df in dfs_a_graficar.items():
    df = df.copy()  
    df["user_ts"] = pd.to_datetime(df["user_ts"], errors="coerce")
    df["user_ts"] = df["user_ts"].dt.tz_localize(None)
    df_filtrado = df[(df["user_ts"] >= fecha_inicio) & (df["user_ts"] <= fecha_fin)].sort_values(by="user_ts")

    if not df_filtrado.empty:
        plt.plot(df_filtrado["user_ts"], df_filtrado["value"], label=nombre, alpha=0.7)


plt.xlabel("Fecha")
plt.ylabel("Valor")
plt.title("Serie de Tiempo de Temperaturas de Preforma y Total Rechazos (1 Nov - 2 Ene)")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()


# -----------------------------------------------------------------Hypothesis3-----------------------------------------------------------------


# CSV file paths
csv_files = {
    "Layer_1": "path_to_layer_1.csv",
    "Layer_3": "path_to_layer_3.csv",
    "Layer_5": "path_to_layer_5.csv",
    "Layer_7": "path_to_layer_7.csv",
    "Layer_9": "path_to_layer_9.csv"
}

# Load each CSV and merge them into a single DataFrame by `user_ts`
df_layers = []
for layer, file in csv_files.items():
    df_temp = pd.read_csv(file)
    df_temp["user_ts"] = pd.to_datetime(df_temp["user_ts"])  # Convert timestamps
    df_temp = df_temp.rename(columns={"value": layer})  # Rename the temperature column
    df_layers.append(df_temp[["user_ts", layer]])  # Keep only `user_ts` and temperature

# Merge datasets by `user_ts`
df = df_layers[0]
for i in range(1, len(df_layers)):
    df = pd.merge(df, df_layers[i], on="user_ts", how="inner")

# Boxplots to visualize the temperature range in each layer
plt.figure(figsize=(12, 6))
sns.boxplot(data=df.drop(columns=["user_ts"]))
plt.title("Temperature Distribution Across Different Layers")
plt.ylabel("Temperature (°C)")
plt.xticks(rotation=45)
plt.show()

# Calculate key statistics
stats = df.drop(columns=["user_ts"]).describe(percentiles=[0.25, 0.5, 0.75])
print("Temperature statistics by layer:")
print(stats)

# Compare temperatures before blower replacement
if "blower_replacement" in df.columns:
    plt.figure(figsize=(12, 6))
    for layer in csv_files.keys():
        sns.lineplot(data=df, x="user_ts", y=layer, label=layer, alpha=0.7)
    plt.axvline(df[df["blower_replacement"] == 1]["user_ts"].min(), color="red", linestyle="dashed", label="Replacement")
    plt.title("Temperatures Before Blower Replacement")
    plt.xlabel("Date")
    plt.ylabel("Temperature (°C)")
    plt.legend()
    plt.show()

# Correlation between temperatures and maintenance frequency
if "maintenance" in df.columns:
    corr_matrix = df.drop(columns=["user_ts"]).corr()
    plt.figure(figsize=(8, 6))
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Correlation Between Temperatures and Maintenance")
    plt.show()